# 📘 Forecasting Evaluation Metrics Lab

This notebook explains **why we need metrics in forecasting**, and how to compute the most common ones: **MSE, RMSE, MAE, and MAPE**.

We will:
1. Define the error term.
2. Introduce each metric with equations and theory.
3. Compute metrics manually and with **Keras**.
4. Compare results.
5. Show how to integrate them in a Keras model.


## 1. Why Do We Need Metrics?

When we build a forecasting or regression model (e.g., ARIMA, LSTM, Linear Regression), predictions $\hat{y}_t$ must be compared against actual values $y_t$.

The error is simply:

$$
e_t = y_t - \hat{y}_t
$$

- $y_t$: actual value at time $t$
- $\hat{y}_t$: predicted (forecasted) value at time $t$
- $e_t$: error (difference between actual and predicted)

Errors can be positive or negative, but to **summarize model performance** across many predictions, we need metrics that capture overall error size. These metrics guide us in comparing models and tuning hyperparameters.

## 2. Mean Squared Error (MSE)

**Definition:**
The **Mean Squared Error (MSE)** is the average of the squared differences between actual and predicted values.

$$
MSE = \frac{1}{n}\sum_{t=1}^{n} (y_t - \hat{y}_t)^2
$$

**Why use it?**
- Squaring ensures positive and negative errors don’t cancel out.
- Large errors are penalized more heavily.
- Commonly used as a loss function in regression tasks (e.g., training neural networks).

**In ML domain:**
- Used in optimization (backpropagation minimizes MSE).
- Good when large deviations are very costly.
- Drawback: units are squared, making interpretation harder.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

# Example data
y_true = np.array([10, 12, 14, 18, 20], dtype=float)
y_pred = np.array([11, 13, 13, 20, 19], dtype=float)

# Manual MSE
mse_manual = np.mean((y_true - y_pred) ** 2)

# Keras MSE
mse_keras = K.eval(tf.keras.metrics.mean_squared_error(y_true, y_pred))

print("Manual MSE:", mse_manual)
print("Keras MSE :", mse_keras)

## 3. Root Mean Squared Error (RMSE)

**Definition:**
The **Root Mean Squared Error (RMSE)** is the square root of the MSE.

$$
RMSE = \sqrt{\frac{1}{n}\sum_{t=1}^{n} (y_t - \hat{y}_t)^2}
$$

**Why use it?**
- Expressed in the same units as the original data, making it easier to interpret than MSE.
- Still penalizes large errors more strongly.

**In ML domain:**
- Common evaluation metric in forecasting competitions (e.g., Kaggle).
- Helps judge the typical error magnitude.
- Sensitive to outliers.

In [ ]:
rmse_manual = np.sqrt(mse_manual)
print("Manual RMSE:", rmse_manual)

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

rmse_keras = K.eval(root_mean_squared_error(y_true, y_pred))
print("Keras RMSE :", rmse_keras)

## 4. Mean Absolute Error (MAE)

**Definition:**
The **Mean Absolute Error (MAE)** is the average of absolute differences between actual and predicted values.

$$
MAE = \frac{1}{n}\sum_{t=1}^{n} |y_t - \hat{y}_t|
$$

**Why use it?**
- Provides average error magnitude directly.
- Less sensitive to outliers compared to MSE.
- Easier to explain to non-technical stakeholders.

**In ML domain:**
- Used when all errors are equally important, regardless of size.
- Popular in demand forecasting and regression problems.
- Does not excessively penalize rare large deviations.

In [ ]:
mae_manual = np.mean(np.abs(y_true - y_pred))
mae_keras = K.eval(tf.keras.metrics.mean_absolute_error(y_true, y_pred))

print("Manual MAE:", mae_manual)
print("Keras MAE :", mae_keras)

## 5. Mean Absolute Percentage Error (MAPE)

**Definition:**
The **Mean Absolute Percentage Error (MAPE)** expresses the error as a percentage of actual values.

$$
MAPE = \frac{100}{n}\sum_{t=1}^{n} \left|\frac{y_t - \hat{y}_t}{y_t}\right|
$$

**Why use it?**
- Intuitive because it shows relative error in % terms.
- Useful when comparing errors across datasets with different scales.

**In ML domain:**
- Widely used in business and finance (easy interpretation for managers).
- Problem: becomes unstable when $y_t$ is close to 0.
- Can exaggerate error if actual values are very small.

In [ ]:
mape_manual = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
mape_keras = K.eval(tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred))

print("Manual MAPE (%):", mape_manual)
print("Keras MAPE (%) :", mape_keras)

## 6. Comparison of Metrics

| Metric | Formula | Pros | Cons |
|--------|---------|------|------|
| **MSE** | $\frac{1}{n}\sum (y-\hat{y})^2$ | Penalizes large errors | Hard to interpret (squared units) |
| **RMSE** | $\sqrt{\frac{1}{n}\sum (y-\hat{y})^2}$ | Same units as data, interpretable | Sensitive to outliers |
| **MAE** | $\frac{1}{n}\sum |y-\hat{y}|$ | Simple, robust to outliers | Less sensitive to big errors |
| **MAPE** | $\frac{100}{n}\sum \left|\frac{y-\hat{y}}{y}\right|$ | Intuitive % error | Problematic if $y \approx 0$ |

## 7. Keras Integration in a Model

When training a Keras model, you can specify metrics like this:

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(1,)),
    tf.keras.layers.Dense(1)
])

model.compile(
    optimizer='adam',
    loss='mse',
    metrics=[
        tf.keras.metrics.MeanSquaredError(name="MSE"),
        tf.keras.metrics.MeanAbsoluteError(name="MAE"),
        tf.keras.metrics.MeanAbsolutePercentageError(name="MAPE"),
        root_mean_squared_error
    ]
)

model.summary()